In [3]:
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import pipeline
transcript=YouTubeTranscriptApi.get_transcript('3pmC0SRynFY')
transcript_ = ' '.join([d['text'] for d in transcript])

In [4]:
print(transcript_)

the true friend once there were two friends a squirrel and a puppy they used to live and play together the squirrel was very sporty and always won the game [Music] the puppy used to feel bad and thought that it was of no use one day it started raining heavily the squirrel was in high spirits he started doing antics but suddenly lost his balance and fell in the rain water he called his friend the puppy for help the puppy came to his rescue [Music] the squirrel climbed on its back and reached a safe place he thanked his friend for saving his life moral of the story is always have confidence in yourself everyone is unique in his own way thanks for watching [Music] you


In [1]:
transcript='''The true friend. Once there were two friends: a squirrel and a puppy. They used to live and play together. The squirrel was very sporty and always won the game. [Music]. The puppy used to feel bad and thought that it was of no use. One day it started raining heavily. The squirrel was in high spirits; he started doing antics but suddenly lost his balance and fell into the rainwater. He called his friend, the puppy, for help. [Music]. The puppy came to his rescue. The squirrel climbed on its back and reached a safe place. He thanked his friend for saving his life. Moral of the story is always to have confidence in yourself; everyone is unique in their own way. Thanks for watching. [Music]. You.'''

In [2]:
import nltk
from nltk import sent_tokenize

using transformer pipeline

In [7]:
summariser = pipeline('summarization')
summary = ''
for i in range(0, (len(transcript_)//1000)+1):
        summary_text = summariser(transcript_[i*1000:(i+1)*1000])[0]['summary_text']
        summary = summary + summary_text + ' '

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Your max_length is set to 142, but your input_length is only 46. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)


In [8]:
print(summary)

 I give a look-alike photo of my favorite hero or cricket star on my facebook so you could keep a photo . People say hey you look like Deepika Padukone, I want to be looked like Katrina Katrina, I. looked like Dhoni, I . want to look like Deepika Padukone and Katrina Katrina Katrina . But when you are not being identified as you as somebody else then you are feeling so proud in that and then you . say nobody takes me seriously why will they take you seriously when you . are not proud about being not proud .  Your individuality that you are somebody and your opinions matter and for that a deep thinking has to happen. reflection has to be part of your routine.   your individuality that  you are  somebody  and  opinions matter . For that, a deep reflection must happen. contemplation has to become part of the routine. 


In [9]:
def evaluation2():
    from rouge import Rouge
    rouge=Rouge()
    scores=rouge.get_scores(transcript_,summary,avg=True)
    print("Rouge-Lsum Precision:", scores["rouge-l"]["p"])
    print("Rouge-Lsum Recall:", scores["rouge-l"]["r"])
    print("Rouge-Lsum F1 Score:", scores["rouge-l"]["f"])
evaluation2()

Rouge-Lsum Precision: 0.584070796460177
Rouge-Lsum Recall: 0.8571428571428571
Rouge-Lsum F1 Score: 0.6947368372847645


In [10]:
def evaluation1():
    from rouge_score import rouge_scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(transcript_,summary)
    for key in scores:
        print(f'{key}: {scores[key]}')
evaluation1()

rouge1: Score(precision=0.8707482993197279, recall=0.5925925925925926, fmeasure=0.7052341597796142)
rouge2: Score(precision=0.773972602739726, recall=0.5255813953488372, fmeasure=0.6260387811634348)
rougeL: Score(precision=0.673469387755102, recall=0.4583333333333333, fmeasure=0.5454545454545454)


Audio for the summary

In [ ]:
import pyttsx3
text_speech = pyttsx3.init()
current_rate = text_speech.getProperty('rate')
new_rate = 150
text_speech.setProperty('rate', new_rate)
text_speech.say(summary)
text_speech.runAndWait()

In [2]:
from transformers import PegasusForConditionalGeneration
from transformers import PegasusTokenizer
from transformers import pipeline

c:\conda\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_name = "google/pegasus-xsum"

# Load pretrained tokenizer
pegasus_tokenizer = PegasusTokenizer.from_pretrained(model_name)
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Create tokens
tokens = pegasus_tokenizer(summary, truncation=True, padding="longest", return_tensors="pt")
encoded_summary = pegasus_model.generate(**tokens)

# Decode summarized text
decoded_summary = pegasus_tokenizer.decode(
      encoded_summary[0],
      skip_special_tokens=True
)
summarizer = pipeline(
    "summarization", 
    model=model_name, 
    tokenizer=pegasus_tokenizer, 
    framework="pt"
)
summary = summarizer(summary, min_length=30, max_length=150)
summary[0]["summary_text"]

In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # You can choose other GPT-2 variants like "gpt2-medium," "gpt2-large," etc.
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Input text to be summarized
input_text = """
Your input text goes here. This is the text you want to generate a summary for.
"""

# Tokenize the input text
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)

# Generate the summary
summary_ids = model.generate(input_ids, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)

# Decode the generated summary back to text
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the generated summary
print("Generated Summary:")
print(summary)
